In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [3]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)  # Camada totalmente conectada
        self.fc2 = nn.Linear(128, 64)      # Camada oculta
        self.fc3 = nn.Linear(64, 10)       # Camada de saída

    def forward(self, x):
        x = x.view(-1, 28*28)  # Achata a imagem para um vetor
        x = torch.relu(self.fc1(x))  # Função de ativação ReLU
        x = torch.relu(self.fc2(x))  # Função de ativação ReLU
        x = self.fc3(x)  # Camada de saída (logits)
        return x


In [4]:
# Transformações para normalizar os dados
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalização
])

# Carregar os dados de treinamento e teste
train_dataset = datasets.MNIST(root='MNIST/', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='MNIST/', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=1000, shuffle=False)


In [5]:
model = SimpleNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [6]:
n_epochs = 5

for epoch in range(n_epochs):
    model.train()
    running_loss = 0.0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{n_epochs}, Loss: {running_loss/len(train_loader)}')


Epoch 1/5, Loss: 0.39166703213379583
Epoch 2/5, Loss: 0.186893996681327
Epoch 3/5, Loss: 0.13394559723243657
Epoch 4/5, Loss: 0.1086913789821062
Epoch 5/5, Loss: 0.09314058356181677


In [7]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')


Accuracy: 96.3%
